In [22]:
from google.colab import files
fread = lambda path: open(path,'r').read()

In [ ]:
#@title Day 1: Secret Entrance
import math

inputs = fread('input.txt').splitlines()

def rotate(pos, d, rotation, part1, part2):
  flip = lambda p : [p%100,(100-p)%100][d=='R']
  pos = flip(pos) + rotation
  return (flip(pos), part1 + (flip(pos) == 0), part2 + math.floor(pos / 100))

acc = (50, 0, 0)
[acc := rotate(acc[0], i[0],int(i[1:]), acc[1], acc[2]) for i in inputs]

print(acc[1])
print(acc[2])

997
5978


In [ ]:
#@title Day 2: Gift Shop
import math, re

inputs = fread('input 2.txt')
inputs = [list(map(int,s.split('-'))) for s in inputs.split(',')]

def bads(lower, upper, pattern):
  some = [i for i in range(lower, upper+1) if re.search(pattern, str(i))]
  return sum([s for s in some if s >= lower and s <= upper])

print(sum([bads(*i,r'^([0-9]+)\1$') for i in inputs]))
print(sum([bads(*i,r'^([0-9]+)\1+$') for i in inputs]))

5398419778
15704845910


In [ ]:
#@title Day 3: Lobby
from itertools import accumulate
import math

inputs = fread('input 3.txt')
inputs = [list(map(int,s)) for s in inputs.splitlines()]

def joltage(line, d):
  def best_idx(prev,j):
    offset = [0,prev+1][prev > -1]
    values = line[offset:len(line)-(d-1-j)]
    return max(range(len(values)), key=values.__getitem__)+offset

  idxs = list(accumulate(range(-1,d), best_idx))[1:]
  return sum([int(math.pow(10,d-1-j)) * line[idxs[j]] for j in range(d)])

print(sum(map(lambda l: joltage(l, 2), inputs)))
print(sum(map(lambda l: joltage(l, 12), inputs)))

17207
170997883706617


In [ ]:
#@title Day 4: Printing Department

inputs = """..@@.@@@@.
@@@.@.@.@@
@@@@@.@.@@
@.@@@@..@.
@@.@@@@.@@
.@@@@@@@.@
.@.@.@.@@@
@.@@@.@@@@
.@@@@@@@@.
@.@.@@@.@."""
inputs = fread('input 4.txt')
inputs = inputs.splitlines()

max_x = len(inputs[0])
max_y = len(inputs)
is_tp = lambda a,x,y: 0 <= x < max_x and 0 <= y < max_y and a[y][x] == '@'

def prune_tp(prev):
  next = []
  count = 0

  for y in range(len(prev)):
    next.append([])

    for x in range(len(prev[0])):
      if not is_tp(prev,x,y):
        next[y].append(prev[y][x])
        continue

      ct = sum([is_tp(prev,i,j) for i in range(x-1,x+2) for j in range(y-1,y+2) if x != i or y != j])

      next[y].append('x' if ct < 4 else '@')
      count += ct < 4

  return (count, next)

def prune_all_tp():
  cont = True
  curr = inputs
  total = 0
  while cont:
    (removed, curr) = prune_tp(curr)
    total += removed
    cont = removed != 0
  return (total, curr)

print(prune_tp(inputs)[0])
print(prune_all_tp()[0])




1564
9401


In [24]:
#@title Day 5-1: Cafeteria
inputs = [s.splitlines() for s in fread('input 5.txt').split('\n\n')];
ranges = [list(map(int,s.split('-'))) for s in inputs[0]]
ids = list(map(int,inputs[1]))
count = sum([any(low <= id <= high for (low,high) in ranges) for id in ids])
print(count)

640


In [36]:
#@title Day 5-2: Cafeteria
inputs = fread('input 5.txt')

inputs = [s.splitlines() for s in inputs.split('\n\n')]
ranges = [list(map(int,s.split('-'))) for s in inputs[0]]
ranges = sorted(ranges, key=lambda r: r[0])

def insert(sorted_list, new_element, i, len):
  if(i >= len):
    return sorted_list + [new_element];
  if(sorted_list[i][0] > new_element[1]):
    return sorted_list[:i] + [new_element] + sorted_list[i:]
  if(sorted_list[i][1] < new_element[0]):
    return insert(sorted_list, new_element, i+1, len)
  return sorted_list[:i] + [[
      min(sorted_list[i][0],new_element[0]),
      max(sorted_list[i][1],new_element[1])
      ]] + sorted_list[i+1:]

merged_ranges = []
for i in range(len(ranges)):
  merged_ranges = insert(merged_ranges, ranges[i], 0, len(merged_ranges))

count = sum([high+1-low for (low,high) in merged_ranges])
print(count)

365804144481581
